In [1]:
!pip install jiwer
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: rapidfuzz
    Found existing installation: rapidfuzz 3.1.1
    Uninstalling rapidfuzz-3.1.1:
      Successfully uninstalled rapidfuzz-3.1.1


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython import display
from jiwer import wer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# data_url = "https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2"
# data_path = keras.utils.get_file("LJSpeech-1.1", data_url, untar=True)
# wavs_path = data_path + "/wavs/"
# metadata_path = data_path + "/metadata.csv"


# # Read metadata file and parse it
# metadata_df = pd.read_csv(metadata_path, sep="|", header=None, quoting=3)
# metadata_df.columns = ["file_name", "transcription", "normalized_transcription"]
# metadata_df = metadata_df[["file_name", "normalized_transcription"]]
# metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)


In [3]:
!pip install keras
!pip install librosa
!pip install matplotlib

In [5]:
# The set of characters accepted in the transcription.
characters = [x for x in "abcdefghijklmnopqrstuvwxyz'?! "]
# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")
# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', ' '] (size =31)


In [6]:
# An integer scalar Tensor. The window length in samples.
frame_length = 256
# An integer scalar Tensor. The number of samples to step.
frame_step = 160
# An integer scalar Tensor. The size of the FFT to apply.
# If not provided, uses the smallest power of 2 enclosing frame_length.
fft_length = 384


def encode_single_sample(wav_file, label):
    ###########################################
    ##  Process the Audio
    ##########################################
    # 1. Read wav file
    file = tf.io.read_file(wavs_path + wav_file + ".wav")
    # 2. Decode the wav file
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    # 3. Change type to float
    audio = tf.cast(audio, tf.float32)
    # 4. Get the spectrogram
    spectrogram = tf.signal.stft(
        audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
    )
    # 5. We only need the magnitude, which can be derived by applying tf.abs
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    # 6. normalisation
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    ###########################################
    ##  Process the label
    ##########################################
    # 7. Convert label to Lower case
    label = tf.strings.lower(label)
    # 8. Split the label
    label = tf.strings.unicode_split(label, input_encoding="UTF-8")
    # 9. Map the characters in label to numbers
    label = char_to_num(label)
    # 10. Return a dict as our model is expecting two inputs
    return spectrogram, label

In [7]:
import tensorflow as tf

# Load the SavedModel
loaded_model = tf.saved_model.load("/kaggle/input/speech-new/speech_new")

In [8]:
loaded_model = tf.keras.models.load_model("/kaggle/input/speech-new/speech_new", compile=False)

In [9]:
import librosa

# Load your own audio file
your_audio_file = '/kaggle/input/audio12/small.wav'  # Replace with the actual path to your audio file
sr = 16000  # Sampling rate, adjust if necessary

# Load the model
loaded_model = keras.models.load_model("/kaggle/input/speech-new/speech_new", compile=False)

# Load and preprocess your audio
def preprocess_audio(audio_file):
    audio, _ = librosa.load(audio_file, sr=sr, mono=True)
    spectrogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    mean = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddev = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - mean) / (stddev + 1e-10)
    return spectrogram

your_audio_spectrogram = preprocess_audio(your_audio_file)

# Make a prediction using the loaded model
def decode_single_prediction(prediction):
    int_prediction = tf.argmax(prediction, axis=-1)
    predicted_text = tf.strings.reduce_join(num_to_char(int_prediction)).numpy().decode("utf-8")
    return predicted_text

your_audio_prediction = loaded_model.predict(tf.expand_dims(your_audio_spectrogram, axis=0))
decoded_prediction = decode_single_prediction(your_audio_prediction[0])

print("Your Audio Prediction:", decoded_prediction)

1/1 [==============================] - 10s 10s/step
Your Audio Prediction: thhiifi   itti  fiffiinn awwwwiieebbu  iit  a  thatt phammeei  petttee  a    thiiminng uuupt  ofllllieeaap  ouretiff aan a ggaaa   a  fiifhe  offf ath  hiff  ffrrik cukerr afiie i  ffi  iit iffii  a  feakkerr  ffaeeedddu  i a ffff  flliiffe  rrrekhherrhi ddakhe ffooui  rrayfhhrrre i  ffi itt ffhifher it  a   iihh lli aatt  addate  ifff  fo abe  rrre ll  ffi  att wwwhhhilllee i uuuuu i  faa houpp  wwhhi at ifaalllllff  the aatt  i aallll aalrreaa i  fff   fivitt  aa wwakkitthhdekkkaa aa ti  aotherrtthaitterr  a rreff annd  i  fiffffer  attt  ii  tthhaatttawwwhhannbeeh  aatt wwwinnkk uuttaaua  fifff  aaiiigggutt aao kkett  vvaat ii a  i  thhheyyh   a  ahhhe  thhek   ivee hhhealaa erri  ff fhaannkkrrraabrraakk iffi a   ttrrekk ifff  ouuuttt  annn effhhek  fekkhia  haatt ffulll hhau  a  hii offiff  rrofeiee    ifff  hhiie diff  baat  hha  braa a  ii  ffiii  ffffthhhiima faarree eerrr aa  o ffa    furri battaa aanmit

In [10]:
# Save the transcription to a file
transcription_filename = 'transcription.txt'
with open(transcription_filename, 'w') as f:
    f.write(decoded_prediction)

In [11]:
print("Transcription saved to:", transcription_filename)

Transcription saved to: transcription.txt
